### download the rolling hindcasts for all GCMs from the Copernicus Climate Datastore

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser

In [3]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [4]:
import numpy as np

In [5]:
HOME = pathlib.Path.home()

In [6]:
CWD = pathlib.Path.cwd()

In [7]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [8]:
sys.path.append('../..')

In [9]:
from ICU_Water_Watch import C3S, domains

### parameters for papermill

In [10]:
provider = 'CDS' # should not change
var_name = 'tprate' # should not change
domain_name = "Water_Watch" # domain name, see below 
# if one wants to download other reforecasts than the ones corresponding to the current month
lag = 0 
# path where to save the hindcasts / reforecasts
gcm_path = f'/media/nicolasf/END19101/ICU/data/{provider}/operational/hindcasts'

### casts the paths to `pathlib.Path` objects 

In [11]:
gcm_path = pathlib.Path(gcm_path)

### mapping GCMs to systems 

#### The mapping between the GCM name and the latest system version available for the forecasts needs to be checked [on the CDS website](https://cds.climate.copernicus.eu/)

In [12]:
from collections import OrderedDict as OD

In [13]:
dict_systems = OD(
                ECMWF=5, # SEAS5, hindcast complete 
                UKMO=600, # note: only for system 14 is hindcast complete
                METEO_FRANCE=8, # note: only for system 7 is hindcast complete
                DWD=21, # GCFS2.1, hindcast complete 
                CMCC=35, # SPSv3.5, hindcast complete 
                NCEP=2,# CFSv2, hindcast complete 
                JMA=2, # CPS2, hindcast complete
                ECCC_GEM_NEMO=2, # GEM-NEMO, hindcast incomplete
                ECCC_CanCM4i=3, # CanCM4i, hindcast incomplete
                )

### get today's date 

In [14]:
date = dateparser.parse('today')

In [15]:
date

datetime.datetime(2022, 1, 18, 13, 13, 10, 856461)

### apply lag 

In [16]:
date = date - relativedelta(months=lag)

In [17]:
print(f"the hindcast data will be downloaded for all re-forecasts initialised in {date:%B}")

the hindcast data will be downloaded for all re-forecasts initialised in January


### generate the dates for the download of the rolling hindcasts 

In [18]:
dates = [datetime(y, date.month, 1) for y in range(1993, 2016 + 1)]

In [19]:
dates

[datetime.datetime(1993, 1, 1, 0, 0),
 datetime.datetime(1994, 1, 1, 0, 0),
 datetime.datetime(1995, 1, 1, 0, 0),
 datetime.datetime(1996, 1, 1, 0, 0),
 datetime.datetime(1997, 1, 1, 0, 0),
 datetime.datetime(1998, 1, 1, 0, 0),
 datetime.datetime(1999, 1, 1, 0, 0),
 datetime.datetime(2000, 1, 1, 0, 0),
 datetime.datetime(2001, 1, 1, 0, 0),
 datetime.datetime(2002, 1, 1, 0, 0),
 datetime.datetime(2003, 1, 1, 0, 0),
 datetime.datetime(2004, 1, 1, 0, 0),
 datetime.datetime(2005, 1, 1, 0, 0),
 datetime.datetime(2006, 1, 1, 0, 0),
 datetime.datetime(2007, 1, 1, 0, 0),
 datetime.datetime(2008, 1, 1, 0, 0),
 datetime.datetime(2009, 1, 1, 0, 0),
 datetime.datetime(2010, 1, 1, 0, 0),
 datetime.datetime(2011, 1, 1, 0, 0),
 datetime.datetime(2012, 1, 1, 0, 0),
 datetime.datetime(2013, 1, 1, 0, 0),
 datetime.datetime(2014, 1, 1, 0, 0),
 datetime.datetime(2015, 1, 1, 0, 0),
 datetime.datetime(2016, 1, 1, 0, 0)]

### path to download the rolling hindcasts 

In [20]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### Now loops over the dates, creates a dictionnary containing the keywords arguments, and pass to dask for parallel download 

In [21]:
list_GCMs = ['ECMWF', 'UKMO', 'METEO_FRANCE', 'DWD', 'CMCC', 'NCEP', 'JMA', 'ECCC_CanCM4i', 'ECCC_GEM_NEMO']

In [ ]:
for date in dates: 
    
    downloaded_files = []

    for GCM in dict_systems.keys():
        
        if 'ECCC' in GCM: 
            
            GCM_API_name = GCM.split('_')[0]
            GCM_path = gcm_path.joinpath(GCM).joinpath(var_name.upper())
            
            print(f"processing ECCC, path should be {str(GCM_path)}")
        
            args = dict(GCM=GCM_API_name,
            system=dict_systems[GCM],
            varname=var_name,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=GCM_path,
            domain=domains.domains['C3S_download'],
            file_format='netcdf',
            level='surface',
            max_retry=3)
        
        else: 

            args = dict(GCM=GCM,
            system=dict_systems[GCM],
            varname=var_name,
            year=date.year,
            month=date.month,
            leadtimes=[1, 2, 3, 4, 5, 6],
            opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
            domain=domains.domains['C3S_download'],
            file_format='netcdf',
            level='surface',
            max_retry=3)
            

        downloaded_files.append(delayed(C3S.download)(**args))

    with ProgressBar():
        downloaded_files = compute(downloaded_files)